In [1]:
import pandas as pd
import math
import igraph
from igraph import Graph

In [2]:
def transformer(x):
    if (x<=20):
        return 0
    if (x<=40):
        return 1
    return 2

x=pd.read_csv('titanic-homework.csv')
del x['Name']
del x['PassengerId']

x['Age']=x['Age'].apply(transformer)

In [114]:
def calc_predefined(full_list):
    values=[full_list[0]/full_list[1], 1-full_list[0]/full_list[1]]
    if (values[0]==0 or values[1]==0):
        return 0
    return sum([-x*math.log(x, 2) for x in values])


def calc_entropy(Y, logs=True):
    possible_values=pd.unique(Y)
    if (possible_values.shape[0]==1):
        if (logs==True):
            print(f'Calculated entropy for "Survived" attribute is equal to 0')
        return 0
    probabilities=[sum(Y==x)/Y.shape[0] for x in possible_values]
    summa=sum([-x*math.log(x, possible_values.shape[0]) for x in probabilities])
    if (logs==True):
            print(f'Calculated entropy for "Survived" attribute is equal to {summa:4f}')
    return summa

def calc_conditional_entropy(Cond, Y):
    values_to_split=pd.unique(Cond)
    res=sum([(Y[Cond==x].shape[0]/Y.shape[0])*calc_entropy(Y[Cond==x], logs=False) for x in values_to_split])
    print(f'Calculated conditional entropy after splitting dataset on column {Cond.name}\n'
          f'Calculated conditional entropy is equal to {res:4f}')
    return sum([(Y[Cond==x].shape[0]/Y.shape[0])*calc_entropy(Y[Cond==x], logs=False) for x in values_to_split])
    
def calc_information_gain(Cond, Y, discrete=True):
    const_entropy=calc_entropy(Y)
    cond_entropy=calc_conditional_entropy(Cond, Y)
    info_gain=const_entropy-cond_entropy
    
    print(f'Information gain info is equal to:\n'
          f' entropy-conditional_entropy={const_entropy:4f}-{cond_entropy:4f}={info_gain:4f}')
    return info_gain

def calc_intrinsic_info(Cond):
    values_to_split=pd.unique(Cond)
    values=[sum(Cond==x)/Cond.shape[0] for x in values_to_split]
    if (values_to_split.shape[0]==1):
        print(f'Intristic info is equal to infinity')
        return 0.00001
    res=sum([-x*math.log(x, 2) for x in values])
    print(f'Intristic info is equal to {res:4f}')
    return res

def solve_continuous(Cond, Y):
    allez=sorted(list(zip(Cond, Y)), key=lambda x:x[0])
    cur_count=[0, 0]
    second_count=[sum(Y==0), sum(Y==1)]
    print("Solution is shown only for best possible split on this column, as\n"
    "otherwise it would be too long")
    const_entropy=calc_entropy(Y)
    wunderbar_point, max_gain_ratio=0, 0
    
    for i, x in enumerate(allez):
        if i==len(allez)-1:
            break
        cur_count[x[1]]+=1
        second_count[x[1]]-=1
        if x[0]==allez[i+1][0]:
            continue
            
        ln=len(allez)
        intristic=calc_predefined([i+1, ln])
        cond_entropy=((i+1)/ln)*calc_predefined([cur_count[0], i+1])+((ln-i-1)/ln)*calc_predefined([second_count[0], ln-i-1])
        info_gain=const_entropy-cond_entropy
        gain_ratio=info_gain/intristic
        
        if (gain_ratio>max_gain_ratio):
            max_gain_ratio, wunderbar_point=gain_ratio, x[0]
            best_data=[intristic, cond_entropy, info_gain, gain_ratio]
    
    intristic=best_data[0]
    cond_entropy=best_data[1]
    info_gain=best_data[2]
    gain_ratio=best_data[3]
    print(f'Intristic info is equal to {intristic:4f}')
    print(f'Calculated conditional entropy after splitting dataset on column {Cond.name} on x<={wunderbar_point}\n'
    f'Calculated conditional entropy is equal to {cond_entropy:4f}')
    print(f'Information gain info is equal to:\n'
          f'entropy-conditional_entropy={const_entropy:4f}-{cond_entropy:4f}={info_gain:4f}')
    print(f'Gain ratio for continuous attribute C is calculated as:\n'
f'information_gain(C, Y)/intristic_info(C)={info_gain:4f}/{intristic:4f}={gain_ratio:4f}')
    
    return wunderbar_point, max_gain_ratio
        
    
def calc_gain_ratio(Cond, Y, discrete=True):
    if discrete==False:
        return solve_continuous(Cond, Y)
    info_gain=calc_information_gain(Cond, Y)
    intr_info=calc_intrinsic_info(Cond)
    res=info_gain/intr_info
    print(f'Gain ratio for discrete attribute C is calculated as:\n'
    f'information_gain(C, Y)/intristic_info(C)={info_gain:4f}/{intr_info:4f}={res:4f}')
    return res
    
def decider(dataset, array_of_discreteness):
    dp=[dataset]
    parents=[-1]
    namez=[]
    
    i=0
    while i<len(dp):
        division=-1
        my_y=dp[i]['Survived']
        print(f'New vertex is processed: this vertex contains\n - {sum(my_y)} survived observations\n - {len(my_y)-sum(my_y)} deceased observations')
        tmp_par=i
        lst_advent=[]
        while tmp_par>0:
            lst_advent.append(namez[tmp_par-1])
            tmp_par=parents[tmp_par]
        lst_advent.reverse()
        print(f'The splits that led to the advent of this vertex were:\n      {lst_advent}')
        
        if (calc_entropy(dp[i]['Survived'], False)==0):
            print(f'############### The vertex is pure, there is nothing more to do in it')
            print('\n')
            i+=1
            continue
        
        mx_ratio=0
        mx_ratio_str=""
        best_division=-1
        
        #-1 - last column - survived
        for j, col in enumerate(dataset.columns[:-1]):
            print(f'Now, the possibility of split on attribute {col} is processed')
            x=calc_gain_ratio(dp[i][col], my_y, array_of_discreteness[j])
            if (array_of_discreteness[j]==False):
                division=x[0]
                x=x[1]
                
            if (array_of_discreteness[j]==False):
                print(f'column: {col}, gain ratio: {x:4f}, split value: {division}')
            else:
                print(f'column: {col}, gain ratio: {x:4f}, attribute discrete')
                
                
            if (x>mx_ratio):
                mx_ratio, mx_ratio_str, best_division=x, col, -1
                if (array_of_discreteness[j]==False):
                    best_division=division
            print()
        if (mx_ratio>0):
            if (best_division==-1):
                values_to_split=pd.unique(dp[i][mx_ratio_str])
                _=[dp.append(dp[i][dp[i][mx_ratio_str]==x]) for x in values_to_split]
                _=[namez.append(f'{mx_ratio_str} = {x}') for x in values_to_split]
            else:
                values_to_split=[-1,-1]
                dp.append(dp[i][dp[i][mx_ratio_str]<=best_division])
                dp.append(dp[i][dp[i][mx_ratio_str]>best_division])
                namez.append(f'{mx_ratio_str} <= {best_division}')
                namez.append(f'{mx_ratio_str} > {best_division}')
                
            _=[parents.append(i) for x in values_to_split]
        if (mx_ratio>0):
            print(f'############### Chosen attribute: {mx_ratio_str}, value of gain: {mx_ratio}')
        else:
            print (f'############### No chosen attribute, end of processing for this subset of dataframe')
        print('\n')
        i+=1
    return [dp, parents, namez]

In [115]:
def create_graph(val):
    nr_vertices=len(val[0])
    g = Graph()
    labels=[f"1:{sum(x['Survived']==1)} / 0:{sum(x['Survived']==0)}" for x in val[0]]
    edges=[(i, val[1][i]) for i in range(1, nr_vertices)]

    g.add_vertices(len(val[0]))
    g.add_edges(edges)
    g.vs["label"]=labels
    g.vs["label_size"]=12
    g.es["label"]=val[2]
    g.es["label_size"]=8
    layout = g.layout_reingold_tilford(root=[0])
    D=1600
    E=80
    visual_style = {"vertex_shape":"circle", "vertex_size":E, "bbox":(D,D), "margin":50}
    return [g, layout, visual_style]
    #igraph.plot(g, layout = layout, **visual_style)

In [116]:
discr=[True]*len(x.columns)
val=decider(x, discr)

New vertex is processed: this vertex contains
 - 40 survived observations
 - 60 deceased observations
The splits that led to the advent of this vertex were:
      []
Now, the possibility of split on attribute Pclass is processed
Calculated entropy for "Survived" attribute is equal to 0.970951
Calculated conditional entropy after splitting dataset on column Pclass
Calculated conditional entropy is equal to 0.889278
Information gain info is equal to:
 entropy-conditional_entropy=0.970951-0.889278=0.081672
Intristic info is equal to 1.370229
Gain ratio for discrete attribute C is calculated as:
information_gain(C, Y)/intristic_info(C)=0.081672/1.370229=0.059605
column: Pclass, gain ratio: 0.059605, attribute discrete

Now, the possibility of split on attribute Sex is processed
Calculated entropy for "Survived" attribute is equal to 0.970951
Calculated conditional entropy after splitting dataset on column Sex
Calculated conditional entropy is equal to 0.579428
Information gain info is equa

Now, the possibility of split on attribute Parch is processed
Calculated entropy for "Survived" attribute is equal to 0.918296
Calculated conditional entropy after splitting dataset on column Parch
Calculated conditional entropy is equal to 0.000000
Information gain info is equal to:
 entropy-conditional_entropy=0.918296-0.000000=0.918296
Intristic info is equal to 1.251629
Gain ratio for discrete attribute C is calculated as:
information_gain(C, Y)/intristic_info(C)=0.918296/1.251629=0.733680
column: Parch, gain ratio: 0.733680, attribute discrete

############### Chosen attribute: Parch, value of gain: 0.7336804366512111


New vertex is processed: this vertex contains
 - 0 survived observations
 - 1 deceased observations
The splits that led to the advent of this vertex were:
      ['Sex = female', 'SibSp = 3', 'Age = 0']
############### The vertex is pure, there is nothing more to do in it


New vertex is processed: this vertex contains
 - 2 survived observations
 - 0 deceased observ

In [117]:
gr=create_graph(val)
pure_graph1=igraph.plot(gr[0], layout=gr[1], **gr[2])

pure_graph1.show()

In [118]:
z=pd.read_csv('titanic-homework.csv')
del z['Name']
del z['PassengerId']

discr=[True if y!='Age' else False for y in z.columns]
val=decider(z, discr)

New vertex is processed: this vertex contains
 - 40 survived observations
 - 60 deceased observations
The splits that led to the advent of this vertex were:
      []
Now, the possibility of split on attribute Pclass is processed
Calculated entropy for "Survived" attribute is equal to 0.970951
Calculated conditional entropy after splitting dataset on column Pclass
Calculated conditional entropy is equal to 0.889278
Information gain info is equal to:
 entropy-conditional_entropy=0.970951-0.889278=0.081672
Intristic info is equal to 1.370229
Gain ratio for discrete attribute C is calculated as:
information_gain(C, Y)/intristic_info(C)=0.081672/1.370229=0.059605
column: Pclass, gain ratio: 0.059605, attribute discrete

Now, the possibility of split on attribute Sex is processed
Calculated entropy for "Survived" attribute is equal to 0.970951
Calculated conditional entropy after splitting dataset on column Sex
Calculated conditional entropy is equal to 0.579428
Information gain info is equa

Calculated conditional entropy is equal to 0.000000
Information gain info is equal to:
 entropy-conditional_entropy=0.918296-0.000000=0.918296
Intristic info is equal to 1.251629
Gain ratio for discrete attribute C is calculated as:
information_gain(C, Y)/intristic_info(C)=0.918296/1.251629=0.733680
column: Parch, gain ratio: 0.733680, attribute discrete

############### Chosen attribute: Parch, value of gain: 0.7336804366512111


New vertex is processed: this vertex contains
 - 0 survived observations
 - 1 deceased observations
The splits that led to the advent of this vertex were:
      ['Sex = female', 'SibSp = 3', 'Age <= 8']
############### The vertex is pure, there is nothing more to do in it


New vertex is processed: this vertex contains
 - 2 survived observations
 - 0 deceased observations
The splits that led to the advent of this vertex were:
      ['Sex = female', 'SibSp = 3', 'Age > 8']
############### The vertex is pure, there is nothing more to do in it


New vertex is pr

In [110]:
fr=create_graph(val)
pure_graph2=igraph.plot(fr[0], layout=fr[1], **fr[2])
pure_graph2.show()